![snap](https://lever-client-logos.s3.amazonaws.com/2bd4cdf9-37f2-497f-9096-c2793296a75f-1568844229943.png)


## Part 1 : Delay Analysis

Author : Youenn PATAT

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 📌 1. Basic analysis and view of data

## For the first page of the dasboard, Home-Intro

In [3]:
data_price = pd.read_csv("https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/get_around_pricing_project.csv", index_col=0)
data_price.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [4]:
mean_rental_per_day = data_price["rental_price_per_day"].mean()
print("The mean rental price per day is:", mean_rental_per_day)

The mean rental price per day is: 121.21453644435267


In [5]:
data = pd.read_excel("https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/get_around_delay_analysis.xlsx")

data.head(10)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN


In [6]:
print("Number of lines:", data.shape[0])

print("Statistics :")
display(data.describe(include="all"))

print("percentage of missing values :")
display(100*data.isnull().sum()/data.shape[0])

Number of lines: 21310
Statistics :


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


percentage of missing values :


rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
delay_at_checkout_in_minutes                  23.294228
previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
dtype: float64

There is a lot af missing values in the 2 last columns, corresponding to the id of previous rental and delay with it if there is a new rental in the same day. That means that most of rentals are made on different days. It also misses 23% of information in the delay checkout.

Taking care of  outliers:

* We have a max delay of 71 000 minutes (too much) and a min delay of -22 000, these values are absurd.

In [7]:
# Count the number of entries with delay_at_checkout_in_minutes > mean + 3*std and < mean - 3*std
mean_delay_checkout = data["delay_at_checkout_in_minutes"].mean()
std_delay_checkout = data["delay_at_checkout_in_minutes"].std()
print("higher bound:", mean_delay_checkout+3*std_delay_checkout)
print("lower bound:", mean_delay_checkout-3*std_delay_checkout)

outliers = data[(data['delay_at_checkout_in_minutes'] > (mean_delay_checkout + 3* std_delay_checkout)) | (data['delay_at_checkout_in_minutes'] < (mean_delay_checkout - 3* std_delay_checkout))]

# Get the count of such entries
num_outliers = len(outliers)

# Display the count
print("Number of ourliers:", num_outliers)

# Filter out and remove the outliers
data = data[(data['delay_at_checkout_in_minutes'] <= (mean_delay_checkout + 3* std_delay_checkout)) & (data['delay_at_checkout_in_minutes'] >= (mean_delay_checkout - 3* std_delay_checkout)) | (data['delay_at_checkout_in_minutes'].isna())]
# We keep the Nan values to keep information of the cancel state of the rental, if not all the cancel state would be removed

# Check the shape of the DataFrame after removing outliers
print("Lines of data after removing outliers:", data.shape[0])

higher bound: 3067.3864217412515
lower bound: -2947.9833873597513
Number of ourliers: 70
Lines of data after removing outliers: 21240


Add a comlumn where checkout delays are categorized, for example:

* $<=$ 0 min for `early or in time`
* $<$ 60 min for `< to 1 hour`
* $<$ 120 min for `1 to 2 hour`
* $<$ 180 min for `2 to 3 hour`
* $<$ 360 min for `3 to 6 hour`
* $<$ 720 min for `6 to 12 hour`
* $<$ 1440 min for `12 to 24 hour`
* $>=$ 1440 min for `1 day or more`
* $=$ Nan for `Unknown`

In [8]:
# Define a function to categorize delays
def categorize_delay(delay):
    if pd.isna(delay):
        return "Unknown"
    elif delay <= 0:
        return "Early or in time"
    elif delay < 60:
        return "< 1 hour"
    elif delay < 120:
        return "1 to 2 hours"
    elif delay < 180:
        return "2 to 3 hours"
    elif delay < 360:
        return "3 to 6 hours"
    elif delay < 720:
        return "6 to 12 hours"
    elif delay < 1440:
        return "12 to 24 hours"
    else:
        return "1 day or more"

# Apply function to create the new column
data["checkout_delay_category"] = data["delay_at_checkout_in_minutes"].apply(categorize_delay)

data.head()


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_category
0,505000,363965,mobile,canceled,NaN,NaN,NaN,Unknown
1,507750,269550,mobile,ended,-81.0,NaN,NaN,Early or in time
2,508131,359049,connect,ended,70.0,NaN,NaN,1 to 2 hours
3,508865,299063,connect,canceled,NaN,NaN,NaN,Unknown
4,511440,313932,mobile,ended,NaN,NaN,NaN,Unknown


In [9]:
# Calculate the value counts of each delay category
delay_counts = data['checkout_delay_category'].value_counts()

# Calculate the percentage of each category
delay_percentages = (delay_counts / delay_counts.sum()) * 100

# Display the result
print(delay_percentages)

checkout_delay_category
Early or in time    32.636535
< 1 hour            23.427495
Unknown             23.370998
1 to 2 hours         8.757062
3 to 6 hours         4.143126
2 to 3 hours         3.775895
6 to 12 hours        1.845574
12 to 24 hours       1.435970
1 day or more        0.607345
Name: count, dtype: float64


First basic views of the data:

In [10]:
#visualisation of the percentage of the mobile vs connect check rental

checkin_counts = data["checkin_type"].value_counts().reset_index()
checkin_counts.columns = ["checkin_type", "count"]

fig = px.pie(checkin_counts, 
             names="checkin_type", 
             values="count", 
             title="Check-in Type Distribution",
             color_discrete_sequence=["#3CB371", "#FFA500"])


fig.show()

In [11]:
#visualisation of the percentage of the mobile vs connect check rental

cancel_counts = data["state"].value_counts().reset_index()
cancel_counts.columns = ["state", "count"]

fig = px.pie(cancel_counts, 
             names="state", 
             values="count", 
             color_discrete_sequence=["#3CB371", "#FFA500"])


fig.show()

In [12]:
# Count occurrences of each category
delay_counts = data["checkout_delay_category"].value_counts().reset_index()
delay_counts.columns = ["Category", "Count"]
delay_counts["Percentage"] = (delay_counts["Count"] / delay_counts["Count"].sum()) * 100

# Define custom colors
custom_colors = {
    "Early or in time": "#FFA500",  # Orange
}

# Assign green as the default color
for category in delay_counts["Category"]:
    if category not in custom_colors:
        custom_colors[category] = "#3CB371"  # Green

# Create a bar chart
fig = px.bar(
    delay_counts, 
    x="Category", 
    y="Count", 
    title="Distribution of Checkout Delays", 
    labels={"Category": "Checkout Delay Category", "Count": "Number of Rentals"},
    color="Category",
    text=delay_counts["Percentage"].apply(lambda x: f"{x:.1f}%"),
    color_discrete_map=custom_colors,
)
fig.update_layout(xaxis_title="", yaxis_title="", showlegend=False)

# Improve layout
# fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray': [
#     "Early or in time", "< 1 hour", "1 to 2 hours", "2 to 3 hours",
#     "3 to 6 hours", "6 to 12 hours", "12 to 24 hours", "1 day or more", "Unknown"
# ]})
fig.show()


There is only 32.6% of rental checkout that are early or in time, without delay. For 23.4% we don't have informations. And the majoruty of delays are less than 2 hours.

In [13]:
# Count occurrences of each category grouped by checkin_type
delay_counts = data.groupby(["checkout_delay_category", "checkin_type"]).size().reset_index(name="Count")
delay_counts["Percentage"] = (delay_counts["Count"] / delay_counts["Count"].sum()) * 100

# Create a grouped bar chart
fig = px.bar(
    delay_counts, 
    x="checkout_delay_category", 
    y="Count", 
    color="checkin_type",
    title="Distribution of Checkout Delays by Check-in Type", 
    labels={"checkout_delay_category": "Checkout Delay Category", "Count": "Number of Rentals", "checkin_type": "Check-in Type"},
    barmode="group",  # Groups bars side by side
    #text="Count",
    text=delay_counts["Percentage"].apply(lambda x: f"{x:.1f}%"),
    color_discrete_sequence=["#FFA500", "#3CB371"]
)

# Improve layout by setting custom order for x-axis
fig.update_layout(xaxis_title="", yaxis_title="")
fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray': [
    "Early or in time", "< 1 hour", "1 to 2 hours", "2 to 3 hours",
    "3 to 6 hours", "6 to 12 hours", "12 to 24 hours", "1 day or more", "Unknown"
]})

fig.show()

There is much more delay problem with mobile checkin type than connect.

---

# For the 2nd page of the dashboard, Analysis & Answer to the problem/questions.

## 📌 2. How often are drivers late for the next check-in? How does it impact the next driver?

* How often the drivers are late for the next check-in:

In [14]:
# Count occurrences of category & group category as simple "late", "in time" or "unknown"
delay_drivers = data["checkout_delay_category"].apply(lambda x: "Early or in time" if x == "Early or in time"
                                                                else "Unkonwn" if x == "Unknown"
                                                                else "Late").value_counts().reset_index()
delay_drivers.columns = ["Category", "Count"]
delay_drivers["Percentage"] = (delay_drivers["Count"] / delay_drivers["Count"].sum()) * 100

# Create a bar chart
fig = px.bar(
    delay_drivers, 
    x="Category", 
    y="Count", 
    labels={"Category": "Checkout Delay Category", "Count": "Number of Rentals"},
    text=delay_drivers["Percentage"].apply(lambda x: f"{x:.1f}%"),
    color_discrete_sequence=["#FFA500"],
)
fig.update_layout(xaxis_title="", yaxis_title="", showlegend=False)

fig.show()


* How does it impact the next driver ?

In [15]:
mean_delay_impact = data["time_delta_with_previous_rental_in_minutes"].mean()
min_delay_impact = data["time_delta_with_previous_rental_in_minutes"].min()
max_delay_impact = data["time_delta_with_previous_rental_in_minutes"].max()

print(f"The average delay that impact the next driver is: {mean_delay_impact} minutes")
print(f"The min delay that impact the next driver is: {min_delay_impact} minutes")
print(f"The max delay that impact the next driver is: {max_delay_impact} minutes")

The average delay that impact the next driver is: 279.2872687704026 minutes
The min delay that impact the next driver is: 0.0 minutes
The max delay that impact the next driver is: 720.0 minutes


In [16]:
delay_impact = data

delay_impact["delta-late_checkout"] = delay_impact["time_delta_with_previous_rental_in_minutes"] - delay_impact["delay_at_checkout_in_minutes"]

#if negative delta - late checkout, it means that the new rental cannot do its check-in
negative_delay_impact = delay_impact[delay_impact["delta-late_checkout"] < 0]

display(negative_delay_impact.head())
print(f"Shape of delay impact dataset : {negative_delay_impact.shape}")

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_category,delta-late_checkout
90,535770,352436,mobile,ended,74.0,524703.0,60.0,1 to 2 hours,-14.0
107,537576,397470,mobile,ended,18.0,539005.0,0.0,< 1 hour,-18.0
148,540479,374684,mobile,ended,12.0,539751.0,0.0,< 1 hour,-12.0
164,541862,382364,mobile,ended,125.0,540607.0,0.0,2 to 3 hours,-125.0
206,543808,369230,mobile,ended,75.0,536315.0,60.0,1 to 2 hours,-15.0


Shape of delay impact dataset : (267, 9)


It shows the number of check-in of the new rentals that cannot have been done due to the previous rental's checkout is late. 

In [17]:
late_checkout = delay_drivers[delay_drivers["Category"] == "Late"]["Count"][0]

nb_problematic_checkin_late = len(negative_delay_impact)
# percentage calculation
problematic_delays_rate = nb_problematic_checkin_late*100/late_checkout
print(f"Among all the delays ({late_checkout}), {round(problematic_delays_rate, 3)}% \n of delays caused problems to the next rental because the checkout\n was made later than the new rental checkin.")




Among all the delays (9344), 2.857% 
 of delays caused problems to the next rental because the checkout
 was made later than the new rental checkin.


In [18]:
# Calculate the average duration of problematic delays
average_problematic_delay = negative_delay_impact['delay_at_checkout_in_minutes'].mean()

# Calculate the average duration of non-problematic delays
average_non_problematic_delay = data[data['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()

# Compare the averages
print("Average Duration of Problematic Delays:", average_problematic_delay)
print("Average Duration of Non-Problematic Delays:", average_non_problematic_delay)

Average Duration of Problematic Delays: 286.7565543071161
Average Duration of Non-Problematic Delays: 146.04173801369862


In [19]:
delay_impact["problematic_delay"] = delay_impact["delta-late_checkout"] < 0
delay_impact["problematic_delay"].value_counts()

problematic_delay
False    20973
True       267
Name: count, dtype: int64

In [20]:
fig = px.histogram(delay_impact, x="problematic_delay", color_discrete_sequence=["#FFA500"],
                )

fig.update_xaxes(
    categoryorder='array',
    categoryarray=["Problematic", "Non-Problematic"]
)
fig.add_annotation(x=3, y=10000,text=f"Avg Delay: {average_problematic_delay:.2f} min",showarrow=False)
fig.add_annotation(x=2, y=10000,text=f"Avg Delay: {average_non_problematic_delay:.2f} min",showarrow=False)
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=[True, False],
        ticktext=["Problematic Delay", "Non Problematic Delay"]
    ),
    xaxis_title="",
    yaxis_title="",
    showlegend=False
)

fig.show()

For the majority of cases, it poses no problem to have delay, but for 2.857% of the case it is problematic for the following rental.

## 📌 3. Which share of our owner’s revenue would potentially be affected by the feature?

In [21]:
# Define the treshold of minimum time between 2 locations (minutes)
thresholds = [30, 60, 90, 120, 180, 360, 720, 1440]  # Example : 1 hour

data["mean_price_per_rental"] = mean_rental_per_day

treshold_data = data
percentage_revenue_impacted = []

for threshold in thresholds:
    treshold_data[f"affected_rentals_{threshold}"] = data["time_delta_with_previous_rental_in_minutes"] <= threshold
    affected_rentals = data[data["time_delta_with_previous_rental_in_minutes"] <= threshold]
    affected_revenue = affected_rentals["mean_price_per_rental"].sum()
    total_revenue = data["mean_price_per_rental"].sum()
    revenue_impact = (affected_revenue / total_revenue) * 100
    percentage_revenue_impacted.append(revenue_impact)

    print(f"Threshold: {threshold} min -> Revenue Impacted: {revenue_impact:.2f}%")

treshold_data.head()

Threshold: 30 min -> Revenue Impacted: 1.89%
Threshold: 60 min -> Revenue Impacted: 2.75%
Threshold: 90 min -> Revenue Impacted: 3.13%
Threshold: 120 min -> Revenue Impacted: 3.77%
Threshold: 180 min -> Revenue Impacted: 4.48%
Threshold: 360 min -> Revenue Impacted: 5.69%
Threshold: 720 min -> Revenue Impacted: 8.65%
Threshold: 1440 min -> Revenue Impacted: 8.65%


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_category,delta-late_checkout,problematic_delay,mean_price_per_rental,affected_rentals_30,affected_rentals_60,affected_rentals_90,affected_rentals_120,affected_rentals_180,affected_rentals_360,affected_rentals_720,affected_rentals_1440
0,505000,363965,mobile,canceled,NaN,NaN,NaN,Unknown,NaN,False,121.214536,False,False,False,False,False,False,False,False
1,507750,269550,mobile,ended,-81.0,NaN,NaN,Early or in time,NaN,False,121.214536,False,False,False,False,False,False,False,False
2,508131,359049,connect,ended,70.0,NaN,NaN,1 to 2 hours,NaN,False,121.214536,False,False,False,False,False,False,False,False
3,508865,299063,connect,canceled,NaN,NaN,NaN,Unknown,NaN,False,121.214536,False,False,False,False,False,False,False,False
4,511440,313932,mobile,ended,NaN,NaN,NaN,Unknown,NaN,False,121.214536,False,False,False,False,False,False,False,False


In [22]:
for threshold in thresholds:
    print(f"treshold {threshold} min", treshold_data[f"affected_rentals_{threshold}"].value_counts())

affected_counts = [treshold_data[f"affected_rentals_{threshold}"].value_counts().get(True, 0) for threshold in thresholds]

treshold 30 min affected_rentals_30
False    20839
True       401
Name: count, dtype: int64
treshold 60 min affected_rentals_60
False    20656
True       584
Name: count, dtype: int64
treshold 90 min affected_rentals_90
False    20576
True       664
Name: count, dtype: int64
treshold 120 min affected_rentals_120
False    20439
True       801
Name: count, dtype: int64
treshold 180 min affected_rentals_180
False    20289
True       951
Name: count, dtype: int64
treshold 360 min affected_rentals_360
False    20031
True      1209
Name: count, dtype: int64
treshold 720 min affected_rentals_720
False    19402
True      1838
Name: count, dtype: int64
treshold 1440 min affected_rentals_1440
False    19402
True      1838
Name: count, dtype: int64


In [24]:
affected_rentals_plot = pd.DataFrame({"Threshold (min)": thresholds, "Affected rentals": affected_counts})

fig = px.line(affected_rentals_plot, x="Threshold (min)", y="Affected rentals", text="Affected rentals",
             title="Number of rentals affected by the treshold",
             color_discrete_sequence=["#3CB371"],)
fig.update_traces(textposition='top center')
fig.update_layout(xaxis_title="", yaxis_title="", showlegend=False)
fig.show()

## 📌 4. How many rentals would be affected by the feature depending on the threshold and scope we choose?

In [26]:
all_affected_list = []
connect_affected_list = []

for threshold in thresholds:
    all_affected = data[data["time_delta_with_previous_rental_in_minutes"] <= threshold].shape[0]
    all_affected_list.append(all_affected)
    connect_affected = data[(data["time_delta_with_previous_rental_in_minutes"] <= threshold) & 
                            (data["checkin_type"] == "connect")].shape[0]
    connect_affected_list.append(connect_affected)

    print(f"Threshold: {threshold} min -> All Rentals Affected: {all_affected}, Connect Rentals Affected: {connect_affected}")


Threshold: 30 min -> All Rentals Affected: 401, Connect Rentals Affected: 181
Threshold: 60 min -> All Rentals Affected: 584, Connect Rentals Affected: 260
Threshold: 90 min -> All Rentals Affected: 664, Connect Rentals Affected: 295
Threshold: 120 min -> All Rentals Affected: 801, Connect Rentals Affected: 345
Threshold: 180 min -> All Rentals Affected: 951, Connect Rentals Affected: 409
Threshold: 360 min -> All Rentals Affected: 1209, Connect Rentals Affected: 517
Threshold: 720 min -> All Rentals Affected: 1838, Connect Rentals Affected: 813
Threshold: 1440 min -> All Rentals Affected: 1838, Connect Rentals Affected: 813


In [27]:
data_affected = pd.DataFrame({ "thresholds" : thresholds,
                 "all_affected" : all_affected_list,
                 "connect_affected" : connect_affected_list})

data_affected

,thresholds,all_affected,connect_affected
0,30,401,181
1,60,584,260
2,90,664,295
3,120,801,345
4,180,951,409
5,360,1209,517
6,720,1838,813
7,1440,1838,813


In [ ]:
fig = px.scatter(data_affected, x='thresholds', y='all_affected',
                 color_discrete_sequence=["#FFA500"],
                 labels={'all_affected': 'All Affected'},)

# Add a line for 'all_affected'
fig.add_trace(go.Scatter(x=data_affected['thresholds'], y=data_affected['all_affected'],
    mode='lines+markers+text', line=dict(color='#FFA500'), name='All Affected', text=data_affected['all_affected']))

fig.add_trace(go.Scatter(x=data_affected['thresholds'], y=data_affected['connect_affected'], 
                    mode='lines+markers+text', marker_color='#3CB371', name='Connect Affected',
                    text=data_affected['connect_affected'],))  # Texte à afficher sur les marqueurs

fig.update_traces(textposition='top center')
 
fig.update_layout(xaxis_title="", yaxis_title="", showlegend=True)

fig.show()


There are less rentals affected with the scope only on connected check-in than all (mobile + connect) check-in. Moreover, as it could be expected, more rentals are impacted with an increasing of the threshold choice.

## 📌 5. How many problematic cases will it solve depending on the chosen threshold and scope?

In [ ]:
solved_cases_all_list = []
solved_cases_connect_list = []

for threshold, i in zip(thresholds, range(len(thresholds))):

    problematic_cases = negative_delay_impact[(negative_delay_impact["delay_at_checkout_in_minutes"] <= threshold)]
    problematic_connectec_case = negative_delay_impact[(negative_delay_impact["delay_at_checkout_in_minutes"] <= threshold) & 
                                                       (negative_delay_impact["checkin_type"] == "connect")]
    total_problems_cases = len(negative_delay_impact)
    total_connect_pb_cases = len(negative_delay_impact[negative_delay_impact["checkin_type"] == "connect"])

    solved_cases = problematic_cases.shape[0]
    solved_cases_all_list.append(solved_cases)
    solved_cases_connect = problematic_connectec_case.shape[0]
    solved_cases_connect_list.append(solved_cases_connect)

    percentage_solved_all = (solved_cases / total_problems_cases) * 100
    percentage_connect_solved = (solved_cases_connect / total_connect_pb_cases) * 100

    print(f"Threshold: {threshold} min -> Problematic Cases for all types of check-in Solved: {solved_cases} that corresponding to {round(percentage_solved_all, 3)}% of problematic cases solved.")
    print(f"Threshold: {threshold} min -> Problematic Cases for connect check-in Solved: {solved_cases_connect} that corresponding to {round(percentage_connect_solved, 3)}% of problematic connect cases solved.")
    print(f"percentage of revenue impacted by threshold: {threshold} -> {percentage_revenue_impacted[i]:.2f} %")
    print("#------------------------------------------------------------------------------------------------------------------------------------------#")


    

Threshold: 30 min -> Problematic Cases for all types of check-in Solved: 45 that corresponding to 16.854% of problematic cases solved.
Threshold: 30 min -> Problematic Cases for connect check-in Solved: 13 that corresponding to 16.25% of problematic connect cases solved.
percentage of revenue impacted by threshold: 30 -> 1.89 %
#------------------------------------------------------------------------------------------------------------------------------------------#
Threshold: 60 min -> Problematic Cases for all types of check-in Solved: 94 that corresponding to 35.206% of problematic cases solved.
Threshold: 60 min -> Problematic Cases for connect check-in Solved: 36 that corresponding to 45.0% of problematic connect cases solved.
percentage of revenue impacted by threshold: 60 -> 2.75 %
#------------------------------------------------------------------------------------------------------------------------------------------#
Threshold: 90 min -> Problematic Cases for all types of che

In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()
# Add line for "All Check-ins"
fig.add_trace(go.Scatter(
    x=thresholds, 
    y=solved_cases_all_list, 
    mode="lines+markers", 
    name="Solved Cases (All Check-ins)", 
    marker=dict(color="#FFA500")
))
# Add line for "Connect Check-ins"
fig.add_trace(go.Scatter(
    x=thresholds, 
    y=solved_cases_connect_list, 
    mode="lines+markers", 
    name="Solved Cases (Connect Check-ins)", 
    marker=dict(color="#3CB371")
))
# Add vertical dashed lines with text annotations
for i, threshold in enumerate(thresholds):
    max_y_value = solved_cases_all_list[i]  # Ensure line stops at "Solved Cases (All Check-ins)"
    
    # Add dashed line from y=0 to y=max_y_value
    fig.add_trace(go.Scatter(
        x=[threshold, threshold],  # Vertical line at threshold
        y=[0, max_y_value],  # Stop at max_y_value
        mode="lines",
        line=dict(color="red", width=1.5, dash="dash"),
        name="Revenue Impact Annotation" if i == 0 else None,  # Show legend only once
        showlegend=(i == 0)
    ))
    # Add text annotation slightly above the dashed line
    fig.add_annotation(
        x=threshold, 
        y=max_y_value + 20,  # Position slightly above the dashed line
        text=f"{percentage_revenue_impacted[i]:.2f}%",  # Format percentage
        showarrow=False,
        font=dict(size=10, color="red"),
        align="center",
    )
fig.update_layout(title="Number of Problematic Cases Solved by Threshold",xaxis_title="",yaxis_title="",showlegend=True)

fig.show()


Now, we can see the problematic cases solved in function of the check-in type (connect or all(mobile+connect)) with the impacted revenue percentage of each threshold. For me the best choice to solve problem without too much economical impact is to choose the threshold of 180 or 360 minutes, for the scope of all check-in type.

---
---